# CPS Reader
##### February 16, 2018 -- @bd_econ

In [6]:
import sys # Check which version of python is being used
print(f'python {sys.version_info[0]}.{sys.version_info[1]}')
import pandas as pd    # Pandas to organize and make calcs
print(f'pandas {pd.__version__}')
import numpy as np     # Numpy for calculations
print(f'numpy {np.__version__}')
import os
import re
import struct
import wquantiles
import calendar
import sqlite3

python 3.6
pandas 0.22.0
numpy 1.14.0


In [7]:
os.chdir('E:/08_Other/Archive/')
conn = sqlite3.connect('C:/Working/econ_data/micro/data/cps.db')

### Data Dictionaries

In [8]:
# Series of interest 
s = ['PWORWGT', 'PWCMPWGT', 'HRHHID', 'HURESPLI', 'HRLONGLK', 'HRHHID2', 
     'PRERNWA', 'PRERNHLY', 'PRTAGE', 'PEAGE', 'PTERNWA', 'PTERNHLY',]

# These series can be stored as categorical later on
s2 = ['HRMONTH', 'PESEX', 'PEMLR', 'PENLFRET', 'PENLFACT', 'PRDISC', 'GESTFIPS',
      'HRMIS', 'PRCOW1', 'PRFTLF', 'PREMPNOT', 'PRCIVLF', 'PEJHRSN', 'PEMJOT',
      'PEEDUCA', 'PRWKSTAT', 'PRDTOCC1', 'GTMETSTA', 'GEMETSTA']   

s = s + s2
d = {}

p = re.compile('\n(\w+)\s+(\d+)\s+(.*?)\t+.*?(\d\d*).*?(\d\d+)')
for y in [2013, 2014, 2015, 2017]:
    # Read the data dictionary file to get the column names and locations
    dd_txt = f'data/January_{y}_Record_Layout.txt'
    dd = open(dd_txt, 'r', encoding='iso-8859-1').read()
    # Regular expression for info of interest based on pattern p
    d[y] = [(i[0], int(i[3]), int(i[1])) for i in p.findall(dd) if i[0] in s]
p = re.compile('\n(\w+)\s+(\d+)\s+(.*?)\s+.*?(\d\d*).*?(\d\d+)')
for v in ['augnov05', 'jan07', 'jan09', 'jan10', 'may12']:
    dd = open(f'data/{v}dd.txt', 'r', encoding='iso-8859-1').read()
    d[v] = [(i[0], int(i[3]), int(i[1])) for i in p.findall(dd) if i[0] in s]
    # Manual touch up of data dictionaries
    d[v][5] = ('HRHHID2', 71, 5)
    d[v][8] = ('PRTAGE', 122, 2)
    d[v][20] = ('PRDTOCC1', 476, 2)

# May 2004 to July 2005 dictionary
dd = open(f'data/may04dd.txt', 'r', encoding='iso-8859-1').read()
d['may04'] = [(i[0], int(i[3]), int(i[1])) for i in p.findall(dd) if i[0] in s]
d['may04'][7] = ('PRTAGE', 122, 2)   
d['may04'][19] = ('PRDTOCC1', 476, 2)
    
# 2003 to May 2004 dictionary    
dd = open(f'data/jan03dd.txt', 'r', encoding='iso-8859-1').read()
d['jan03'] = [(i[0], int(i[3]), int(i[1])) for i in p.findall(dd) if i[0] in s]
d['jan03'][6] = ('GTMETSTA', 105, 1)
d['jan03'][7] = ('PRTAGE', 122, 2) 
d['jan03'][19] = ('PRDTOCC1', 476, 2)
    
# 1998 to 2002 dictionary    
p = re.compile('D (\w+)\s+(\d{1,2})\s+(\d+)\s+')
dd = open(f'data/jan98dd.asc', 'r', encoding='iso-8859-1').read()
d['jan98'] = [(i[0], int(i[2]), int(i[1])) for i in p.findall(dd) if i[0] in s]
d['jan98'][20] = ('PRERNHLY', 520, 4)
d['jan98'][21] = ('PRERNWA', 527, 8)
d['jan98'][6] = ('GTMETSTA', 105, 1)

# Manually note years where data dictionary does not change
d[2016] = d[2015]
d[2018] = d[2017]
d[2010] = d['jan10']
d[2011] = d['jan10']
d[2009] = d['jan09']
d[2008] = d['jan07']
d[2007] = d['jan07']
d[2006] = d['augnov05']
d[2003] = d['jan03']
d[2002] = d['jan98']
d[2001] = d['jan98']
d[2000] = d['jan98']
d[1999] = d['jan98']
d[1998] = d['jan98']

In [ ]:
#p = re.compile('\n(\w+)\s+(\d+)\s+(.*?)\s+.*?(\d\d*).*?(\d\d+)')
#dd = open(f'data/may04dd.txt', 'r', encoding='iso-8859-1').read()
#[(i[0], int(i[3]), int(i[1])) for i in p.findall(dd) if i[0] in s]

In [ ]:
#len(d['may04'])

In [9]:
print(dd)

       CURRENT POPULATION SURVEY,
       BASIC MONTHLY DATA DICTIONARY
             January 1998

 **Note: For value set descriptions not 
   found in this file, go to 
   http://www.bls.census.gov/cps/basic/datadict/199801/puf98dd.htm

DATA        SIZE  BEGIN
 
D HRHHID     15      1
T Household-identifier,scrambled
     Household-identifier,scrambled Edited 
     Universe: All Households in Sample 

D HRMONTH     2     16
T Household-month of interview
     Month Of Interview Valid Entries 
V          1 .January
V          2 .February
V          3 .March
V          4 .April
V          5 .May
V          6 .June
V          7 .July
V          8 .August
V          9 .September
V         10 .October
V         11 .November
V         12 .December

D HRYEAR4     4     18
T Household-4 digit year of interview
U All Households 
V  1998:2999 .Year

D HURESPLI    2     22
T Household-line number of current respondent
     Line number of the current respondent. 
     Valid Entries 
V         -3 .

### Read a data dictionary into the notebook

In [ ]:
#print(open(f'data/jan10dd.txt', 'r', encoding='iso-8859-1').read())

### Unzipper

In [ ]:
#from zipfile import ZipFile
#for file in [f for f in os.listdir('data/') if f.endswith('pub.zip')]:
#    with ZipFile(f'data/{file}', 'r') as zip_ref:
#        zip_ref.extractall('data/')

### Convert .cps to .dat

In [ ]:
#for file in [f for f in os.listdir('data/') if f.endswith('cps')]:
#    os.rename(f'data/{file}', f'data/{file[:-4]}.dat')

### Some tools

In [ ]:
def struct_constr(fieldspecs):
    """Build a fast way to read fixed width files"""
    unpack_len = 0
    unpack_fmt = ""
    for fieldspec in fieldspecs:
        start = fieldspec[1] - 1
        end = start + fieldspec[2]
        if start > unpack_len:
            unpack_fmt += str(start - unpack_len) + "x"
        unpack_fmt += str(end - start) + "s"
        unpack_len = end
    field_indices = range(len(fieldspecs))
    return struct.Struct(unpack_fmt).unpack_from

def fwf_to_list(file, unpacker, l):
    with open(f'data/{file}', 'r', encoding='utf-8') as f:
        for line in f:
            raw_fields = unpacker(line.encode())
            if int(raw_fields[-1]) > 0:   # Filter out rows with no weight
                l.append(tuple(int(x) for x in raw_fields))
        
                
def list_to_pickle(l, fieldspecs, year):
    """Store list as pickled file for fast reading later"""
    df = pd.DataFrame(l, columns = [f[0] for f in fieldspecs]
                     ).apply(pd.to_numeric,downcast='unsigned')
    for col in [f[0] for f in fieldspecs if f in s2]: 
        df[col] = df[col].astype('category').cat.as_ordered()
    df.to_pickle(f'data/cps_{year}.pkl')
    
def list_to_df(l, fieldspecs, year):
    """Store list as pandas dataframe"""
    df = pd.DataFrame(l, columns = [f[0] for f in fieldspecs]
                     ).apply(pd.to_numeric,downcast='unsigned')
    for col in [f[0] for f in fieldspecs if f in s2]: 
        df[col] = df[col].astype('category').cat.as_ordered()
    return df

def df_to_sql(df, year):
    """Send pandas df to sql database"""
    df.to_sql(f'cps_{year}', conn, if_exists='replace')
    
def list_to_sql(l, fieldspecs, year):
    """Store list into sql database"""
    df = pd.DataFrame(l, columns = [f[0] for f in fieldspecs]
                     ).apply(pd.to_numeric,downcast='unsigned')
    for col in [f[0] for f in fieldspecs if f in s2]: 
        df[col] = df[col].astype('category').cat.as_ordered()
    df.to_sql(f'cps_{year}', conn, if_exists='replace')

def special_years(year, fs1, fs2, sm):
    """Handle cases where dictionary is split in middle of year"""
    if year == year:
        fieldspecs = fs1
        unpacker = struct_constr(fieldspecs)

        l = []
        # Fill list with monthly data from each monthly file
        for file in [f'{calendar.month_name[i].lower()[:3]}{str(year)[2:]}pub.dat' for i in range(1,sm)]:
            fwf_to_list(file, unpacker, l)
        df2 = list_to_df(l, fieldspecs, year)
                        
        fieldspecs = fs2
        unpacker = struct_constr(fieldspecs)

        # Fill list with monthly data from each monthly file
        for file in [f'{calendar.month_name[i].lower()[:3]}{str(year)[2:]}pub.dat' for i in range(sm,13)]:
            fwf_to_list(file, unpacker, l)
        df3 = list_to_df(l, fieldspecs, year)
        df = df2.append(df3)
        df_to_sql(df, year)

### Read monthly data as annual dataframe

In [ ]:
def monthly_to_annual(year):
    """Read monthly files and store as one annual file"""
    # Field specs (field name, start pos (1-relative), len)
    if year != 2012 and year !=2005 and year !=2004: 
        fieldspecs = d[year]
        unpacker = struct_constr(fieldspecs)

        l = []
        # Fill list with monthly data from each monthly file
        for file in [f for f in os.listdir('data/') if f.endswith(f'{str(year)[-2:]}pub.dat')]:
            fwf_to_list(file, unpacker, l)
        list_to_sql(l, fieldspecs, year)
        
    if year == 2012: special_years(2012, d[2011], d['may12'], 5)
    if year == 2005: special_years(2005, d['may04'], d['augnov05'], 8)
    if year == 2004: special_years(2004, d['jan03'], d['may04'], 5)

### Applying the function to selected years

In [ ]:
for year in range(1998, 1999):
    monthly_to_annual(year)

### Benchmarking

In [ ]:
# Usual Weekly Earnings 2017 Q1

sql_exec = ("select PRERNWA, PWORWGT from cps_2017 "
            "where (HRMONTH between 1 and 3) and (PRERNWA > -1) "
            "and (PRTAGE >=16) and (PRFTLF = 1)")

df = pd.read_sql(sql_exec, conn)
print('2017 Q1 Usual Weekly Earnings: ${0:,.2f}'.format(
    wquantiles.median(df['PRERNWA'], df['PWORWGT']) / 100.0))

In [ ]:
# Usual Weekly Earnings 2017 Q1
df = pd.read_pickle(f'data/cps_2017.pkl')
df = df[(df['PRERNWA'] > -1) & (df['PRTAGE'] >= 16) & 
        (df['PRFTLF'] == 1) & (df['HRMONTH'].isin([1, 2, 3]))]
        
print('2017 Q1 Usual Weekly Earnings: ${0:,.2f}'.format(
    wquantiles.median(df['PRERNWA'], df['PWORWGT']) / 100.0))

In [ ]:
# Prime age epop for women, Jan 2017 = 71.3 (LNU02300062)
df = pd.read_pickle(f'data/cps_2008.pkl')
df = df[(df['HRMONTH'] == 1) & (df['PESEX'] == 2) 
         & (df['PRTAGE'].isin(range(25,55)))]
empl = np.where(df['PREMPNOT']==1, 1, 0)
epop = np.average(empl.astype(float), weights=df['PWCMPWGT']) * 100
print(f'January 2008: {round(epop, 1)}')

In [ ]:
# Number of people not in the labor force
df = pd.read_pickle(f'data/cps_2017.pkl')
wc = float(max(df['HRMONTH'].unique()) * 10000000)
print(f"{y}: {(df[df['PEMLR'] >= 5]['PWORWGT'].sum() / wc) :,.0f}")

In [ ]:
df = pd.read_pickle(f'data/cps_2003.pkl')
wc = float(max(df['HRMONTH'].unique()) * 10000000)
df[(df['PENLFACT'] == 3) & (df['PRTAGE'].isin(range(25,35)))].groupby('PESEX')['PWCMPWGT'].sum() / wc 

### Making some calculations

In [ ]:
for year in range(1998, 2019):
    df = pd.read_pickle(f'data/cps_{year}.pkl')
    df = df[df['PRTAGE'].isin(range(25,35))]
    tot = df['PWCMPWGT'].sum()
    empl = df[df['PREMPNOT'] == 1]['PWCMPWGT'].sum()
    epop = empl / tot
    print(f'{year}: {epop * 100: 3.1f}')

In [ ]:
for year in range(1998, 2019):
    for month in range(1, 13):
        ex = (f"select PREMPNOT, PWCMPWGT from cps_{year} "
              f"where (HRMONTH = {month}) and (PRTAGE between 25 and 34)")
        df = pd.read_sql(ex, conn)
        empl = np.where(df['PREMPNOT']==1, 1, 0)
        epop = np.average(empl.astype(float), weights=df['PWCMPWGT']) * 100
        print(f'{month} {year}: {round(epop, 1)}')

In [ ]:
pd.read_sql("SELECT * FROM cps_2005", conn)